In [37]:
# Set sun, earth position
sun_mass = 1.98892 * 10**30
earth_mass = 5.9742 * 10**24
sun_to_earth = 1 # AU, check to make sure gravity is in AU

# Functions for computing gravity vectors
Gsc = 1.361 # kW/m2
A = 1 # m**2     # sail area
C_SRP    = 2.00 # SRP coefficent, C_SRP = 1 for perfectly absorbing, C_SRP = 2 for perfectly reflective  solar sail
# FN= 2GsccAcos2(α)≈(9μN/m2) cos2(α)

dim = 2 # 2 or 3d sim, starting with 2

# The gravitational constant G
G = 6.67428e-11

# Assumed scale: 100 pixels = 1AU.
AU = (149.6e6 * 1000)     # 149.6 million km, in meters.
SCALE = 250 / AU

# constants for inertial properties and torques
global r_ss, rp_ss, A_ss, Ap_ss, hp_ss, lp_ss, P_ss
r_ss = 0.56 #m, 
rp_ss = 0.565 #m
A_ss = 1 #m**2, 
Ap_ss = 0.02 #m**2
hp_ss = 10E-2 #m  
lp_ss = 20E-2 #m.
m_mp = 0.001 # kg (1gram)
m_ss = 0.0016 # kg (1.6grams)
mp_ss = 0.000032 # kg (32mg) 
P_ss = 9E-6 # uPa. (estimate)

# equations for roll flag control
# thetap_r = 2*math.asin(dyp_r/2/lp_ss)
# alphap_r = alpha - thetap_r

# Fp_s = f(alpha_roll, theta) = P_ss*Ap_ss*cos(alphap_r)**3
def Fp_s(alpha_r):
    return P_ss*Ap_ss*(math.cos(alpha_r)**2)

# F_s = f(alpha)  = P_ss*A_ss* cos(alpha)**3
def F_s(alpha_r):
    return P_ss*A_ss*(math.cos(alpha_r)**2)

# dx,dz in 0-10mm
#def T_yy(angle):
    # T_yy = Fp_s * rp_ss * math.sin(thetap_r) # = I_yy*theta_dot_dot

def T_zz(angle, dx):
    # T_zz = F_s*dx #  I_zz*alpha_dot_dot
    return F_s(angle)*dx

def T_xx(angle, dz):
    # T_xx = F_s*dz #  I_xx*alpha_dot_dot
    return F_s(angle)*dz

def inertials(dx,dy,dz,m_mp = 0.001, m_ss = 0.0016):
    ratio = m_mp/(m_ss+m_mp)
    x_ss = dx*ratio
    x_mp = dx*(1-ratio)
#     y_ss = dy*ratio
    y_tot = 2 #.05 # 2
    y_ss = y_tot*ratio
#     y_mp = dy*(1-ratio)
    y_mp = y_tot*(1-ratio)
    z_ss = dz*ratio
    z_mp = dz*(1-ratio)

    I_yy = 1 / 2 * m_ss * r_ss **2+ m_ss * (x_ss **2+ z_ss **2)+mp_ss * rp_ss **2+1 / 12 * mp_ss * hp_ss **2 + m_mp * (x_mp **2+z_mp**2)
    I_zz = 1 / 4 * m_ss * r_ss **2+m_ss * (x_ss **2+ y_ss **2)+m_mp * (x_mp**2+y_mp **2)
    I_xx = 1 / 4 * m_ss * r_ss **2+m_ss * (z_ss **2+ y_ss **2)+m_mp * (z_mp**2+y_mp **2)
    return I_xx, I_yy, I_zz

f = F_s(0)
n = 100
alpha = np.linspace(-np.pi/2,np.pi/2,n)
dx = np.linspace(-0.01,0.01,n)
dz = np.linspace(-0.01,0.01,n)
xx,zz = np.meshgrid(dx,dz)
rates = np.zeros((n,n,3))
for i, x in enumerate(dx):
    for j, a in enumerate(alpha):
        f = F_s(a)
        z = 0
        ix, iy, iz = inertials(x,0,z)
        tx = f *x #z
        ty = 0
        tz = f* z #x
        ox = tx/ix
        oy = 0
        oz = tz/iz
        rates[i,j,0] = ox
        rates[i,j,2] = oz 

        
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
        z=rates[:,:,0],
        x=alpha,
        y=dx,
        colorscale='Viridis')
                )

fig.update_layout(
    title='Angular Rates vs Sun Angle vs Input',
    xaxis_title='cone angle (rad)',
    yaxis_title='input motion (m)')

fig.show()

In [19]:

class SolarSail
    def __init__(self, mass, Ixx, Iyy, Izz):
        self.mass = mass
        self.Ixx = Ixx
        self.Iyy = Iyy
        self.Izz = Izz
        
    def pos2force(self, cone, clock=None):    
        # FN= 2GsccAcos2(α)≈(9μN/m2) cos2(α)
        # RIGHT NOW FORCE IS AT CENTER OF MASS
        # SRP on Sail (A_SRP is a constant 1.0 m**2 right now but through control of
        # sail orientation, we can have this vary between 0 <= A_SRP <= 1.
        # avec_SRP_Sail     = (1/m) * P_SRP(r_Sun_Sail) * C_SRP * A_SRP * rvec_Sun_Sail/r_Sun_Sail;

        
    
    def cone2torques(self, cone, clock=None):

    def gravity(self, obj):
        # Compute the distance of the other body.
        sx, sy = self.px, self.py
        ox, oy = other.px, other.py
        dx = (ox-sx)
        dy = (oy-sy)
        d = math.sqrt(dx**2 + dy**2)

        # Report an error if the distance is zero; otherwise wepll
        # get a ZeroDivisionError exception further down.
        if d == 0:
            raise ValueError("Collision between objects %r and %r"
                             % (self.name, other.name))

        # Compute the force of attraction
        f = G * self.mass * other.mass / (d**2)

        # Compute the direction of the force.
        theta = math.atan2(dy, dx)
        fx = math.cos(theta) * f
        fy = math.sin(theta) * f
        return fx, fy


SyntaxError: invalid syntax (<ipython-input-19-2325ef0aceb2>, line 1)

In [20]:
# PID controller for roll (add noise to simulation)
class PID():
    def __init__(self, desired,
                 kp, ki, kd,
                 ilimit, dt, outlimit=np.inf,
                 samplingRate=0, cutoffFreq=-1,
                 enableDFilter=False):

        # internal variables
        self.error = 0
        self.error_prev = 0
        self.integral = 0
        self.deriv = 0
        self.out = 0

        # constants
        self.desired = desired
        self.kp = kp
        self.ki = ki
        self.kd = kd

        # limits integral growth
        self.ilimit = ilimit

        # limits ridiculous actions. Should set to variance
        self.outlimit = outlimit

        # timee steps for changing step size of PID response
        self.dt = dt
        self.samplingRate = samplingRate  # sample rate is for filtering

        self.cutoffFreq = cutoffFreq
        self.enableDFilter = enableDFilter

        if cutoffFreq != -1 or enableDFilter:
            raise NotImplementedError('Have not implemnted filtering yet')

    def reset(self):
        # internal variables
        self.error = 0
        self.error_prev = 0
        self.integral = 0
        self.deriv = 0

    def update(self, measured):

        # init
        self.out = 0.

        # update error
        self.error_prev = self.error

        # calc new error
        self.error = self.desired - measured

        # proportional gain is easy
        self.out += self.kp * self.error

        # calculate deriv term
        self.deriv = (self.error - self.error_prev) / self.dt

        # filtter if needed (DT function_)
        if self.enableDFilter:
            print('Do Filter')
            self.deriv = self.deriv

        # calcualte error value added
        self.out += self.deriv * self.kd

        # accumualte normalized eerror
        self.integral = self.error * self.dt

        # limitt the integral term
        if self.ilimit != 0:
            self.integral = np.clip(self.integral, -self.ilimit, self.ilimit)

        self.out += self.ki * self.integral

        # limitt the total output
        if self.outlimit != 0:
            self.out = np.clip(self.out, -self.outlimit, self.outlimit)

        return self.out

In [85]:
# loop of computing forward dynamics
import numpy as np
# need info about initial condition of orbit
mu = 3.986004418*10**14 # SGP earth

# v = np.sqrt(mu/r)

# low orbit: 180-20000 km
# mid orbit: 2000-35780 km
# high orbit: >35,780 km

# https://en.wikipedia.org/wiki/Orbital_speed

# some targets in solar sail angle at different places in orbit
# XY Plane / polar works. 
# 
targets_visual =  np.array([[90, -90], 
            [135, -67.5], 
            [180, -45],
            [225, -22.5],
            [270, 0],
            [315, 22.5],
            [0, 45], 
            [45, 67.5], 
            [89.9, 90]])

targets_actual = targets_visual - np.array([90,0])
print(targets_actual)
            
          
print([(angle-90)/2 - 90 for angle in [90, 180, 270, 360, 450]])
    
            

[[ 0.00e+00 -9.00e+01]
 [ 4.50e+01 -6.75e+01]
 [ 9.00e+01 -4.50e+01]
 [ 1.35e+02 -2.25e+01]
 [ 1.80e+02  0.00e+00]
 [ 2.25e+02  2.25e+01]
 [-9.00e+01  4.50e+01]
 [-4.50e+01  6.75e+01]
 [-1.00e-01  9.00e+01]]
[-90.0, -45.0, 0.0, 45.0, 90.0]


In [127]:
def grav(x,y, mass1, mass2):
    d = np.sqrt(x**2+y**2)
    f = G * mass1 * mass2 / (d**2)

    # Compute the direction of the force.
    theta = math.atan2(y, x)
    fx = math.cos(theta) * f
    fy = math.sin(theta) * f
    return fx, fy
    
def earth2clock(angle):
    # convert from earth position to desired clock angle.
    # to start with, will just use this as the actual angle
    angle = np.rad2deg(angle)
    while angle < 90:
        angle += 360
    return (angle-90)/2 - 90
    
def clock2force(angle):
    # for now assumes the y distance from the sun line is 0
    force_mag = P_ss*Ap_ss*(math.cos(angle)**2)
    return force_mag*math.cos(angle), force_mag*math.sin(angle)
    
    
    # visualize
# from https://medium.com/analytics-vidhya/simulating-the-solar-system-with-under-100-lines-of-python-code-5c53b3039fc6
import plotly
import plotly.graph_objects as go

init_orbit = 30000E3
# 1.496e+11 m in 1 AU
e = np.array([0,0])
# r = np.array([0, -6.684587e-5])*1.496e+11 # xy pos sail relative to earth, 10000km below in this diagram
r = np.array([0, -init_orbit]) # xy pos sail relative to earth, 10000km below in this diagram
s = np.array([-1, 0])*1.496e+11 # sun is 1 AU to the "left" of earth

mu = 3.986004418*10**14 # SGP earth

m_ss = 0.001
m_earth =  5.972 * 10**24 

n_steps = 100
t_vec = np.linspace(0,100000, n_steps)
v0 = np.sqrt(np.abs(mu/r[1]))
v0_sail = np.array([v0,0])
clock0 = 0



x,y = r
v = v0_sail
dt = 600 # seconds

# arrays to store and plot
Xs = []
Ys = []
F_sail = []
F_grav = []
Vs = []

for t in range(n_steps): #t_vec:
    print(f"===== Time {t*dt}(s) =====")
    sun2sail = r + np.array([x,y])
    earth_angle = math.atan2(y, x)
    clock_angle = earth2clock(earth_angle) # current hard code "control"

    print(f"- Angle from earth to sail about orbit circle {earth_angle}")
    print(f"- Earth angle {np.rad2deg(earth_angle)}")
    print(f"- Clock angle {clock_angle}")
    
    f_ss = np.array(clock2force(np.deg2rad(clock_angle)))
    f_ge = np.array(grav(x,y,m_ss,m_earth))
    f_gs = np.array([0,0 ]) # no sun gravity yet
    F_sail.append(f_ss)
    F_grav.append(F_grav)
                  
    x = x + v[0]*dt
    y = y + v[1]*dt
    Xs.append(x)
    Ys.append(y)
    f_tot = f_ss + f_ge + f_gs
    print(f"- XY force sail {f_ss}")
    print(f"- XY force grav {f_ge}")
    print(f"- XY pos [{x/1000},{y/1000}] km")
                  
    v = v + (f_tot/m_ss)*dt
    Vs.append(v)
                  
    print(f"- Velocity at this step {v}")
    print(f"- Radius {np.sqrt(x**2+y**2)/1000} km")
    print()


===== Time 0(s) =====
- Angle from earth to sail about orbit circle -1.5707963267948966
- Earth angle -90.0
- Clock angle 0.0
- XY force sail [1.8e-07 0.0e+00]
- XY force grav [ 2.71183067e-20 -4.42875557e-04]
- XY pos [2187.0540234754144,-30000.0] km
- Velocity at this step [3645.19803913 -265.7253344 ]
- Radius 30079.6144473562 km

===== Time 600(s) =====
- Angle from earth to sail about orbit circle -1.4980232654695538
- Earth angle -85.83041072381114
- Clock angle 2.0847946380944222
- XY force sail [1.79642801e-07 6.53946460e-09]
- XY force grav [ 3.20307374e-05 -4.39368260e-04]
- XY pos [4374.172846950829,-30159.43520064] km
- Velocity at this step [3664.52426722 -529.34236669]
- Radius 30474.98842849009 km

===== Time 1200(s) =====
- Angle from earth to sail about orbit circle -1.4267656531711288
- Earth angle -81.74765028093188
- Clock angle 4.126174859534046
- XY force sail [1.78603954e-07 1.28845055e-08]
- XY force grav [ 6.16012484e-05 -4.24733755e-04]
- XY pos [6572.88740728

In [138]:
import plotly.graph_objects as go
Vs = np.stack(Vs)

xm = -2*init_orbit
xM = 2*init_orbit
ym = -2*init_orbit
yM = 2*init_orbit
# Create figure
fig = go.Figure(
    data=[go.Scatter(x=np.stack(Xs).tolist(), y=np.stack(Ys).tolist(),
                     name="Traj.",
                     mode="lines",
                     line=dict(width=2, color="blue")),
          go.Scatter(x=[0], y=[0],
                     name="Earth",
                     mode="markers",
                     marker=dict(size=20, color="black")),
          go.Scatter(x=np.stack(Xs).tolist(), y=np.stack(Ys).tolist(),
                     name="curve",
                     showlegend=False,
                     mode="lines",
                     line=dict(width=2, color="blue"))
          ],
    layout=go.Layout(width=600, height=600,
                     xaxis=dict(range=[xm, xM], autorange=False, zeroline=False),
                     yaxis=dict(range=[ym, yM], autorange=False, zeroline=False),
                     title="Animating Solar Sail Orbit + Forces",
                     hovermode="closest",
                     updatemenus=[dict(type="buttons",
                                       buttons=[{
                                            "args": [None, {"frame": {"duration": 50, "redraw": False},
                                                            "fromcurrent": True, "transition": {"duration": 30,
                                                                                                "easing": "quadratic-in-out"}}],
                                            "label": "Play",
                                            "method": "animate"
                                        },
                                        {
                                            "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                                              "mode": "immediate",
                                                              "transition": {"duration": 0}}],
                                            "label": "Pause",
                                            "method": "animate"
                                        },
#                                        {
#                                         "args": [None, {"frame": {"duration": 50, "redraw": True},
#                                                         "fromcurrent": False, "transition": {"duration": 30,
#                                                                                             "easing": "quadratic-in-out"}}],
#                                         "label": "Reset",
#                                         "method": "animate"
#                                     },
                                    ],
                                       direction= "left",
                                    pad= {"r": 10, "t": 70},
                                    x= 0.1,
                                    y= 0,
                                    )]
                    ), # was none

    frames=[go.Frame(
        data=[go.Scatter(
            x=[Xs[k], Xs[k]+1000*Vs[k,0], None, Xs[k], Xs[k]+1000*Vs[k,0]],
            y=[Ys[k], Ys[k]+1000*Vs[k,1], None, Ys[k], Ys[k]+1000*Vs[k,1]],
#             x=[xx[k], xend[k], None, xx[k], xnoe[k]],
#             y=[yy[k], yend[k], None, yy[k], ynoe[k]],
            mode="lines",
            line=dict(color="red", width=4))
        ], 
        name=str(k)) for k in range(n_steps)]
)

def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }
sliders = [
            {
                "pad": {"b": 10, "t": 60},
                "len": 0.7,
                "x": 0.2,
                "y": 0,
                "steps": [
                    {
                        "args": [[f.name], frame_args(0)],
                        "label": str(k),
                        "method": "animate",
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]
                                
fig.update_layout(sliders=sliders)

fig.show()

In [106]:
print(np.stack(Vs))

[[ 3645.10083913   -26.57253344]
 [ 3645.30534202   -53.14290929]
 [ 3645.7033936    -79.70407041]
 [ 3646.29466505  -106.24897232]
 [ 3647.07865377  -132.77059319]
 [ 3648.05468457  -159.2619436 ]
 [ 3649.22191134  -185.71607637]
 [ 3650.57931904  -212.1260961 ]
 [ 3652.12572623  -238.48516862]
 [ 3653.85978787  -264.78653022]
 [ 3655.77999858  -291.02349668]
 [ 3657.88469623  -317.18947194]
 [ 3660.17206585  -343.27795664]
 [ 3662.64014396  -369.2825562 ]
 [ 3665.28682311  -395.19698869]
 [ 3668.10985677  -421.01509226]
 [ 3671.10686447  -446.73083226]
 [ 3674.2753372   -472.33830791]
 [ 3677.61264299  -497.83175866]
 [ 3681.11603277  -523.20557002]
 [ 3684.7826463   -548.45427907]
 [ 3688.60951836  -573.57257945]
 [ 3692.593585    -598.55532597]
 [ 3696.73168986  -623.39753874]
 [ 3701.02059071  -648.09440687]
 [ 3705.45696586  -672.64129172]
 [ 3710.03742074  -697.03372969]
 [ 3714.75849446  -721.26743461]
 [ 3719.61666627  -745.33829962]
 [ 3724.60836214  -769.24239871]
 [ 3729.72

In [ ]:

# Create figure
fig = go.Figure(
    data=[go.Scatter(x=x, y=y,
                     mode="lines",
                     line=dict(width=2, color="blue")),
          go.Scatter(x=x, y=y,
                     mode="lines",
                     line=dict(width=2, color="blue"))],
    layout=go.Layout(
        xaxis=dict(range=[xm, xM], autorange=False, zeroline=False),
        yaxis=dict(range=[ym, yM], autorange=False, zeroline=False),
        title_text="Kinematic Generation of a Planar Curve", hovermode="closest",
        updatemenus=[dict(type="buttons",
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None])])]),
    frames=[go.Frame(
        data=[go.Scatter(
            x=[xx[k]],
            y=[yy[k]],
            mode="markers",
            marker=dict(color="red", size=10))])

        for k in range(N)]
)

fig.show()

In [5]:
import numpy as np
import torch
import torch.optim as optim
import math
import gym
from gym import spaces, logger
from gym.utils import seeding


class RigidEnv(gym.Env):
    """
    Source:
        This file is created by Nathan Lambert, adapted from a model from Somil Bansal
    Observation: 
        Type: Box(12)
        Num	Observation                 Min         Max
        0	x-pos                       -10         10      (meters)
        1	y-pos                       -10         10      (meters)
        2	z-pos                       -10         10      (meters)
        3	x-vel                       -Inf        Inf     (meters/sec)
        4   y-vel                       -Inf        Inf     (meters/sec)
        5   z-vel                       -Inf        Inf     (meters/sec)
        6   roll                        -180        180     (degrees)
        7   pitch                       -90         90      (degrees)
        8   yaw                         -180        180     (degrees)
        9   omega_x                     -Inf        Inf     (rad/s^2)
        10  omega_y                     -Inf        Inf     (rad/s^2)
        11  omega_z                     -Inf        Inf     (rad/s^2)
    """

    def __init__(self, dt=.01, x_noise=0.0, u_noise=0): #0.01 noise
        self.x_dim = 12
        self.u_dim = 4
        self.dt = dt
        self.x_noise = x_noise

        # simulate ten steps per return
        self.dt = self.dt

        # Setup the state indices
        self.idx_xyz = [0, 1, 2]
        self.idx_xyz_dot = [3, 4, 5]
        self.idx_ptp = [6, 7, 8]
        self.idx_ptp_dot = [9, 10, 11]

        self.seed()
        self.viewer = None
        self.state = None
        self.steps_beyond_done = None

    def seed(self, seed=None, inertial=False):
        self.np_random, seed = seeding.np_random(seed)
        if inertial:
            self.Ixx = self.Ixx * self.np_random.uniform(low=0.85, high=1.15, size=1)[0]
            self.Iyy = self.Iyy * self.np_random.uniform(low=0.85, high=1.15, size=1)[0]

        return [seed]

    def step(self, forces, torques):
        # assert self.action_space.contains(u), "%r (%s) invalid" % (u, type(u))

        # We need to convert from upright orientation to N-E-Down that the simulator runs in
        # For reference, a negative thrust of -mg/4 will keep the robot stable
        u = self.pwm_thrust_torque(pwm)
        state = self.state
        self.last_state = state

        dt = self.dt
        u0 = u
        x0 = state
        idx_xyz = self.idx_xyz
        idx_xyz_dot = self.idx_xyz_dot
        idx_ptp = self.idx_ptp
        idx_ptp_dot = self.idx_ptp_dot

        m = self.m
        L = self.L
        Ixx = self.Ixx
        Iyy = self.Iyy
        Izz = self.Izz
        g = self.g
        
        # Cross term, direct term for the three torques
        Tx = np.array([Iyy / Ixx - Izz / Ixx, L / Ixx])
        Ty = np.array([Izz / Iyy - Ixx / Iyy, L / Iyy])
        Tz = np.array([Ixx / Izz - Iyy / Izz, 1. / Izz])

        # # Add noise to input
        # u_noise_vec = np.random.normal(
        #     loc=0, scale=self.u_noise, size=(self.u_dim))
        # u = u+u_noise_vec

        # Array containing the forces
        # This assumes the force all comes directly out of the plane
        # Third column of Qb/i (converts body forces to inertial frame)
        Fxyz = np.zeros(3)
        Fxyz[0] = -1 * (math.cos(x0[idx_ptp[0]]) * math.sin(x0[idx_ptp[1]]) * math.cos(x0[idx_ptp[2]]) + math.sin(x0[idx_ptp[0]]) * math.sin(x0[idx_ptp[2]])) * u0[0] / m
        Fxyz[1] = -1 * (math.cos(x0[idx_ptp[0]]) * math.sin(x0[idx_ptp[1]]) * math.sin(x0[idx_ptp[2]]) - math.sin(x0[idx_ptp[0]]) * math.cos(x0[idx_ptp[2]])) * u0[0] / m
        Fxyz[2] = g - 1 * (math.cos(x0[idx_ptp[0]]) * math.cos(x0[idx_ptp[1]])) * u0[0] / m

        # Compute the torques
        t0 = np.array([x0[idx_ptp_dot[1]] * x0[idx_ptp_dot[2]], u0[1]])
        t1 = np.array([x0[idx_ptp_dot[0]] * x0[idx_ptp_dot[2]], u0[2]])
        t2 = np.array([x0[idx_ptp_dot[0]] * x0[idx_ptp_dot[1]], u0[3]])
        Txyz = np.array([Tx.dot(t0), Ty.dot(t1), Tz.dot(t2)])

        x1 = np.zeros(12)
        x1[idx_xyz_dot] = x0[idx_xyz_dot] + dt * Fxyz
        x1[idx_ptp_dot] = x0[idx_ptp_dot] + dt * Txyz
        x1[idx_xyz] = x0[idx_xyz] + dt * x0[idx_xyz_dot]
        x1[idx_ptp] = x0[idx_ptp] + dt * self.pqr2rpy(x0[idx_ptp], x0[idx_ptp_dot])

        # makes states less than 1e-12 = 0
        x1[abs(x1) < 1e-15] = 0
        self.state = x1
        state = x1

        # Add noise component
        x_noise_vec = np.random.normal(
            loc=0, scale=self.x_noise, size=(self.x_dim))

        self.state += x_noise_vec

        obs = self.get_obs()
        reward = self.get_reward(obs, u)
        done = self.get_done(obs)

        return obs, reward, done, {}

    def get_obs(self):
        raise NotImplementedError("Subclass must implement this function")

    def set_state(self, x):
        self.state = x

    def reset(self, safe=False):
        x0 = np.array([0, 0, 0])
        v0 = self.np_random.uniform(low=-0.01, high=0.01, size=(3,))
        # ypr0 = self.np_random.uniform(low=-0.0, high=0.0, size=(3,))
        if safe:
            ypr0 = self.np_random.uniform(low=-0.01, high=0.01, size=(3,))
            ypr0 = self.np_random.uniform(low=-0.00, high=0.00, size=(3,))
            w0 = self.np_random.uniform(low=-0.00, high=0.00, size=(3,))
        else:
            ypr0 = self.np_random.uniform(low=-np.pi/16., high=np.pi/16., size=(3,))
            w0 = self.np_random.uniform(low=-0.01, high=0.01, size=(3,))

        ypr0[-1] = 0 # 0 out yaw

        self.state = np.concatenate([x0, v0, ypr0, w0])
        self.last_state = self.state
        self.steps_beyond_done = None
        self.interal = 0

        return self.get_obs()

    def get_done(self, state):
        # Done is pitch or roll > 35 deg
        max_a = np.deg2rad(45)
        d = (abs(state[1]) > max_a) or (abs(state[0]) > max_a)
        return d

    def pqr2rpy(self, x0, pqr):
        rotn_matrix = np.array([[1., math.sin(x0[0]) * math.tan(x0[1]), math.cos(x0[0]) * math.tan(x0[1])],
                                [0., math.cos(x0[0]), -math.sin(x0[0])],
                                [0., math.sin(x0[0]) / math.cos(x0[1]), math.cos(x0[0]) / math.cos(x0[1])]])
        return rotn_matrix.dot(pqr)


